### Cuidados!
 
1. Whatsapp não gosta de nenhum tipo de automação
2. Isso não é o uso da API oficial do Whatsapp, o próprio whatsapp tem uma API oficial. Se o seu objetivo é fazer envio em massa ou criar aqueles robozinhos que respondem automaticamente no whatsapp, então use a API oficial
3. Este código é destinado apenas para fins de demonstração e aprendizado.


### Dito isso, bora automatizar o envio de mensagens no Whatsapp

- Vou usar Selenium
- Usar o https://web.whatsapp.com/send?phone=numero&text=texto 


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import urllib  # Responsável pela conversão do texto
import time
import os
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager  # Certifique-se de importar o ChromeDriverManager corretamente

# Configuração do serviço e opções do Chrome
service = webdriver.chrome.service.Service(ChromeDriverManager().install())
options = webdriver.ChromeOptions()
options.add_argument(rf"--user-data-dir={os.path.expanduser('~')}\AppData\Local\Google\Chrome\User Data\Perfil Whatsapp Selenium")
navegador = webdriver.Chrome(service=service, options=options)

# Abrir o WhatsApp Web
navegador.get("https://web.whatsapp.com")

# Esperar a tela do WhatsApp carregar
while len(navegador.find_elements(By.XPATH, '//*[@id="app"]/div/div[2]/div[3]')) < 1:
    time.sleep(1)
time.sleep(2)  # O WhatsApp já carregou

# Ler os dados da tabela
tabela = pd.read_excel("Envios.xlsx")
display(tabela[['nome', 'mensagem', 'arquivo', 'telefone']])

# Loop para enviar as mensagens
for linha in tabela.index:
    # Obter dados da tabela
    nome = tabela.loc[linha, "nome"]
    mensagem = tabela.loc[linha, "mensagem"]
    arquivo = tabela.loc[linha, "arquivo"]
    telefone = tabela.loc[linha, "telefone"]

    # Substituir "Fulano" pelo nome e fazer o encode do texto
    texto = mensagem.replace("Fulano", nome)
    texto = urllib.parse.quote(texto)

    # Gerar o link de envio de mensagem
    link = f"https://web.whatsapp.com/send?phone={telefone}&text={texto}"
    navegador.get(link)

    try:
        # Esperar a tela do WhatsApp carregar
        WebDriverWait(navegador, 20).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="app"]/div/div[2]/div[3]'))
        )

        # Verificar se o número de telefone é válido
        invalid_number_element_xpath = '//*[@id="app"]/div/span[2]/div/span/div/div/div/div'
        if not navegador.find_elements(By.XPATH, invalid_number_element_xpath):
            # Esperar o botão de envio carregar e enviar a mensagem
            send_button = WebDriverWait(navegador, 20).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="main"]/footer/div[1]/div/span[2]/div/div[2]/div[2]/button/span'))
            )
            send_button.click()
            time.sleep(2)

            # Enviar arquivo, se houver
            if arquivo != "N":
                caminho_completo = os.path.abspath(f"arquivos/{arquivo}")
                navegador.find_element(By.XPATH, '//*[@id="main"]/footer/div[1]/div/span[2]/div/div[1]/div[2]/div/div/div/span').click()
                navegador.find_element(By.XPATH, '//*[@id="main"]/footer/div[1]/div/span[2]/div/div[1]/div[2]/div/span/div/ul/div/div[1]/li/div/input').send_keys(caminho_completo)
                time.sleep(5)
                navegador.find_element(By.XPATH, '//*[@id="app"]/div/div[2]/div[2]/div[2]/span/div/div/div/div[2]/div/div[2]/div[2]/div/div/span').click()
                time.sleep(5)

    except Exception as e:
        print(f"Erro ao tentar enviar mensagem para {nome}: {e}")

# Fechar o navegador após o script terminar
navegador.quit()
